# Fetch Assessment Data Exploration

In [5]:
# Function to extract date
def extract_date(value):
  """
    Extract date from a nested dictionary.
  """
  if isinstance(value, dict) and '$date' in value:
        return value['$date']
  return value

In [8]:
# Function to convert epoch into datetime
def convert_epoch_to_datetime(df, epoch_columns):
    """
    Converts specified epoch timestamp columns to datetime format.
    """
    for col in epoch_columns:
      # Determine whether the epoch is in seconds or milliseconds
      unit = 's' if df[col].iloc[0] < 10**10 else 'ms'  # 's' for seconds, 'ms' for milliseconds

      # Convert and store the result in a new column with a "_datetime" suffix
      df[f'{col}_datetime'] = pd.to_datetime(df[col], unit=unit)
    return df

In [1]:
# Import necessary libraries and upload receipts data
import pandas as pd
import json
from google.colab import files

uploaded = files.upload()


Saving receipts.json to receipts (1).json


## Receipts Data Schema

* _id: uuid for this receipt
* bonusPointsEarned: Number of bonus points that were awarded upon receipt completion
* bonusPointsEarnedReason: event that triggered bonus points
* createDate: The date that the event was created
* dateScanned: Date that the user scanned their receipt
* finishedDate: Date that the receipt finished processing
* modifyDate: The date the event was modified
* pointsAwardedDate: The date we awarded points for the transaction
* pointsEarned: The number of points earned for the receipt
* purchaseDate: the date of the purchase
* purchasedItemCount: Count of number of items on the receipt
* rewardsReceiptItemList: The items that were purchased on the receipt
* rewardsReceiptStatus: status of the receipt through receipt validation and processing
* totalSpent: The total amount on the receipt
* userId: string id back to the User collection for the user who scanned the receipt




In [2]:
filename = list(uploaded.keys())[0]  # Geting the file name

# Using pandas.read_json to handle JSONL files
df = pd.read_json(filename, lines=True)

# Converting JSON data to a pandas DataFrame
df.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    float64
 14  userId                  

In [4]:
# Data Transformation

df['id'] = df['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else x)
df.drop(columns=['_id'], inplace=True)



df = df.map(extract_date)
df.head()

,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,id
0,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575
1,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b
2,5.0,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5.0,1.609632e+12,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a
3,5.0,All-receipts receipt bonus,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,5.0,1.609632e+12,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f
4,5.0,All-receipts receipt bonus,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,5.0,1.609601e+12,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561


In [5]:
df['rewardsReceiptItemList'] = df['rewardsReceiptItemList'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

In [7]:
dict_column = 'rewardsReceiptItemList'
# Extracting all values from the dictionaries and expand them into new columns
df_extracted = df[dict_column].apply(lambda x: pd.Series(x) if isinstance(x, dict) else pd.Series())

# Combining the extracted columns with the original DataFrame
for col in df_extracted.columns:
    df[f"{dict_column}_{col}"] = df_extracted[col]

In [7]:
df.head()

,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,...,rewardsReceiptItemList_itemNumber,rewardsReceiptItemList_needsFetchReviewReason,rewardsReceiptItemList_originalMetaBriteQuantityPurchased,rewardsReceiptItemList_pointsEarned,rewardsReceiptItemList_targetPrice,rewardsReceiptItemList_competitiveProduct,rewardsReceiptItemList_userFlaggedDescription,rewardsReceiptItemList_deleted,rewardsReceiptItemList_priceAfterCoupon,rewardsReceiptItemList_metabriteCampaignId
0,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.0,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5.0,1.609632e+12,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,All-receipts receipt bonus,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,5.0,1.609632e+12,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,All-receipts receipt bonus,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,5.0,1.609601e+12,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 47 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   bonusPointsEarned                                          544 non-null    float64
 1   bonusPointsEarnedReason                                    544 non-null    object 
 2   createDate                                                 1119 non-null   int64  
 3   dateScanned                                                1119 non-null   int64  
 4   finishedDate                                               568 non-null    float64
 5   modifyDate                                                 1119 non-null   int64  
 6   pointsAwardedDate                                          537 non-null    float64
 7   pointsEarned                                               609 non-null    float64
 8   purchase

In [8]:
# Table split based on the data model
# Receipt
receipt_df = df[['id', 'userId', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
                 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'pointsEarned', 'purchaseDate'
                 , 'purchasedItemCount', 'rewardsReceiptStatus', 'totalSpent']].copy() #df.iloc[:, :15]

# receiptItemList
receipt_list_df = df.iloc[:, 13:]

In [9]:
# Transform epoch datetime into datetime
epoch_list = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']

for col in receipt_df.columns:
    if col in epoch_list:  #checking if it is a epoch column
        unit = 's' if receipt_df[col].iloc[0] < 10**10 else 'ms'  # 's' for seconds, 'ms' for milliseconds

        # Convert and store the result in a new column with a "_datetime" suffix
        try:
            receipt_df[f'{col}_datetime'] = pd.to_datetime(receipt_df[col], unit=unit, errors='coerce')
        except OverflowError:
            # If overflow occurs, try dividing by 1000 to convert from milliseconds to seconds
            if unit == 'ms':
                receipt_df[f'{col}_datetime'] = pd.to_datetime(receipt_df[col] / 1000, unit='s', errors='coerce')
            else:
                raise


In [11]:
# renaming for readability
receipt_df = receipt_df.rename(columns={'id': 'receipt_id', 'userId': 'user_id', 'createDate': 'createDate_epoch',
                                        'dateScanned':'dateScanned_epoch', 'finishedDate':'finishedDate_epoch', 'modifyDate': 'ModifyDate_epoch',
                                        'pointsAwardedDate':'pointsAwardedDate_epoch', 'purchaseDate':'purchaseDate_epoch', 'createDate_datetime':'createDate',
                                        'dateScanned_datetime':'dateScanned', 'finishedDate_datetime': 'finishedDate', 'modifyDate_datetime': 'modifyDate',
                                        'pointsAwardedDate_datetime':'pointsAwardedDate', 'purchaseDate_datetime':'purchaseDate'})
receipt_df.head()

,receipt_id,user_id,bonusPointsEarned,bonusPointsEarnedReason,createDate_epoch,dateScanned_epoch,finishedDate_epoch,ModifyDate_epoch,pointsAwardedDate_epoch,pointsEarned,purchaseDate_epoch,purchasedItemCount,rewardsReceiptStatus,totalSpent,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,purchaseDate
0,5ff1e1eb0a720f0523000575,5ff1e1eacfcf6c399c274ae6,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,5.0,FINISHED,26.0,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,2021-01-03 00:00:00
1,5ff1e1bb0a720f052300056b,5ff1e194b6a9d73a3a9f1052,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,2.0,FINISHED,11.0,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,2021-01-02 15:24:43
2,5ff1e1f10a720f052300057a,5ff1e1f1cfcf6c399c274b0b,5.0,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5.0,1.609632e+12,1.0,REJECTED,10.0,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,2021-01-03 00:00:00
3,5ff1e1ee0a7214ada100056f,5ff1e1eacfcf6c399c274ae6,5.0,All-receipts receipt bonus,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,5.0,1.609632e+12,4.0,FINISHED,28.0,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,2021-01-03 00:00:00
4,5ff1e1d20a7214ada1000561,5ff1e194b6a9d73a3a9f1052,5.0,All-receipts receipt bonus,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,5.0,1.609601e+12,2.0,FINISHED,1.0,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,2021-01-02 15:25:06


In [13]:
# Final table split based on the data model

receipt_df = receipt_df[['receipt_id', 'user_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate_epoch', 'createDate','dateScanned_epoch',
                         'dateScanned', 'finishedDate_epoch', 'finishedDate', 'ModifyDate_epoch', 'modifyDate','pointsAwardedDate_epoch', 'pointsAwardedDate',
                         'purchaseDate_epoch', 'purchaseDate','pointsEarned', 'purchasedItemCount', 'rewardsReceiptStatus', 'totalSpent']]

In [14]:
receipt_df.head()

,receipt_id,user_id,bonusPointsEarned,bonusPointsEarnedReason,createDate_epoch,createDate,dateScanned_epoch,dateScanned,finishedDate_epoch,finishedDate,ModifyDate_epoch,modifyDate,pointsAwardedDate_epoch,pointsAwardedDate,purchaseDate_epoch,purchaseDate,pointsEarned,purchasedItemCount,rewardsReceiptStatus,totalSpent
0,5ff1e1eb0a720f0523000575,5ff1e1eacfcf6c399c274ae6,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,2021-01-03 15:25:31,1609687531000,2021-01-03 15:25:31,1.609688e+12,2021-01-03 15:25:31,1609687536000,2021-01-03 15:25:36,1.609688e+12,2021-01-03 15:25:31,1.609632e+12,2021-01-03 00:00:00,500.0,5.0,FINISHED,26.0
1,5ff1e1bb0a720f052300056b,5ff1e194b6a9d73a3a9f1052,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,2021-01-03 15:24:43,1609687483000,2021-01-03 15:24:43,1.609687e+12,2021-01-03 15:24:43,1609687488000,2021-01-03 15:24:48,1.609687e+12,2021-01-03 15:24:43,1.609601e+12,2021-01-02 15:24:43,150.0,2.0,FINISHED,11.0
2,5ff1e1f10a720f052300057a,5ff1e1f1cfcf6c399c274b0b,5.0,All-receipts receipt bonus,1609687537000,2021-01-03 15:25:37,1609687537000,2021-01-03 15:25:37,NaN,NaT,1609687542000,2021-01-03 15:25:42,NaN,NaT,1.609632e+12,2021-01-03 00:00:00,5.0,1.0,REJECTED,10.0
3,5ff1e1ee0a7214ada100056f,5ff1e1eacfcf6c399c274ae6,5.0,All-receipts receipt bonus,1609687534000,2021-01-03 15:25:34,1609687534000,2021-01-03 15:25:34,1.609688e+12,2021-01-03 15:25:34,1609687539000,2021-01-03 15:25:39,1.609688e+12,2021-01-03 15:25:34,1.609632e+12,2021-01-03 00:00:00,5.0,4.0,FINISHED,28.0
4,5ff1e1d20a7214ada1000561,5ff1e194b6a9d73a3a9f1052,5.0,All-receipts receipt bonus,1609687506000,2021-01-03 15:25:06,1609687506000,2021-01-03 15:25:06,1.609688e+12,2021-01-03 15:25:11,1609687511000,2021-01-03 15:25:11,1.609688e+12,2021-01-03 15:25:06,1.609601e+12,2021-01-02 15:25:06,5.0,2.0,FINISHED,1.0


In [29]:
receipt_list_df.columns

Index(['user_id', 'receipt_id', 'barcode', 'description', 'finalPrice',
       'itemPrice', 'needsFetchReview', 'partnerItemId',
       'preventTargetGapPoints', 'quantityPurchased', 'userFlaggedBarcode',
       'userFlaggedNewItem', 'userFlaggedPrice', 'userFlaggedQuantity',
       'originalMetaBriteBarcode',
       'rewardsReceiptItemList_originalMetaBriteDescription',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'brandCode', 'competitorRewardsGroup',
       'discountedItemPrice', 'originalReceiptItemText', 'itemNumber',
       'needsFetchReviewReason', 'originalMetaBriteQuantityPurchased',
       'pointsEarned', 'targetPrice', 'competitiveProduct',
       'userFlaggedDescription', 'deleted', 'priceAfterCoupon',
       'metabriteCampaignId'],
      dtype='object')

In [22]:
# renaming for readability
receipt_list_df = receipt_list_df.rename(columns={'userid': 'user_id',
                                                  'id': 'receipt_id',
                                                  'rewardsReceiptItemList_barcode': 'barcode',
                                                  'rewardsReceiptItemList_description': 'description',
                                                  'rewardsReceiptItemList_finalPrice': 'finalPrice',
                                                  'rewardsReceiptItemList_itemPrice': 'itemPrice',
                                                  'rewardsReceiptItemList_needsFetchReview': 'needsFetchReview',
                                                  'rewardsReceiptItemList_partnerItemId': 'partnerItemId',
                                                  'rewardsReceiptItemList_preventTargetGapPoints':'preventTargetGapPoints',
                                                  'rewardsReceiptItemList_quantityPurchased': 'quantityPurchased',
                                                  'rewardsReceiptItemList_userFlaggedBarcode': 'userFlaggedBarcode',
                                                  'rewardsReceiptItemList_userFlaggedNewItem': 'userFlaggedNewItem',
                                                  'rewardsReceiptItemList_userFlaggedPrice': 'userFlaggedPrice',
                                                  'rewardsReceiptItemList_userFlaggedQuantity': 'userFlaggedQuantity',
                                                  'rewardsReceiptItemList_originalMetaBriteBarcode': 'originalMetaBriteBarcode',
                                                  'rewardsReceiptItemList_oriinalMetaBriteDescription': 'originalMetaBriteDescription',
                                                  'rewardsReceiptItemList_pointsNotAwardedReason': 'pointsNotAwardedReason',
                                                  'rewardsReceiptItemList_pointsPayerId': 'pointsPayerId',
                                                  'rewardsReceiptItemList_rewardsGroup': 'rewardsGroup',
                                                  'rewardsReceiptItemList_rewardsProductPartnerId': 'rewardsProductPartnerId',
                                                  'rewardsReceiptItemList_brandCode': 'brandCode',
                                                  'rewardsReceiptItemList_competitorRewardsGroup': 'competitorRewardsGroup',
                                                  'rewardsReceiptItemList_discountedItemPrice': 'discountedItemPrice',
                                                  'rewardsReceiptItemList_originalReceiptItemText': 'originalReceiptItemText',
                                                  'rewardsReceiptItemList_itemNumber': 'itemNumber',
                                                  'rewardsReceiptItemList_needsFetchReviewReason': 'needsFetchReviewReason',
                                                  'rewardsReceiptItemList_originalMetaBriteQuantity': 'originalMetaBriteQuantity',
                                                  'rewardsReceiptItemList_pointsEarned': 'pointsEarned',
                                                  'rewardsReceiptItemList_targetPrice': 'targetPrice',
                                                  'rewardsReceiptItemList_competitiveProduct': 'competitiveProduct',
                                                  'rewardsReceiptItemList_userFlaggedDescription': 'userFlaggedDescription',
                                                  'rewardsReceiptItemList_deleted': 'deleted',
                                                  'rewardsReceiptItemList_priceAfterCoupon': 'priceAfterCoupon',
                                                  'rewardsReceiptItemList_metabriteCampaignId': 'metabriteCampaignId'})
receipt_list_df.head()

,userId,receipt_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,...,itemNumber,needsFetchReviewReason,rewardsReceiptItemList_originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,userFlaggedDescription,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
receipt_list_df = receipt_list_df.rename(columns={'rewardsReceiptItemList_originalMetaBriteDescription': 'originalMetaBriteDescription'})

In [25]:
receipt_list_df.head()

,user_id,receipt_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,...,itemNumber,needsFetchReviewReason,rewardsReceiptItemList_originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,userFlaggedDescription,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
receipt_df.to_csv('receipts.csv', index=False)

# Download the transformed receipts table as a CSV file
from google.colab import files
files.download('receipts.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
receipt_list_df.head()

,userId,id,rewardsReceiptItemList_barcode,rewardsReceiptItemList_description,rewardsReceiptItemList_finalPrice,rewardsReceiptItemList_itemPrice,rewardsReceiptItemList_needsFetchReview,rewardsReceiptItemList_partnerItemId,rewardsReceiptItemList_preventTargetGapPoints,rewardsReceiptItemList_quantityPurchased,...,rewardsReceiptItemList_itemNumber,rewardsReceiptItemList_needsFetchReviewReason,rewardsReceiptItemList_originalMetaBriteQuantityPurchased,rewardsReceiptItemList_pointsEarned,rewardsReceiptItemList_targetPrice,rewardsReceiptItemList_competitiveProduct,rewardsReceiptItemList_userFlaggedDescription,rewardsReceiptItemList_deleted,rewardsReceiptItemList_priceAfterCoupon,rewardsReceiptItemList_metabriteCampaignId
0,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
receipt_list_df.to_csv('receipt_list_df.csv', index=False)

# Download the transformed receipts list table as a CSV file
from google.colab import files
files.download('receipt_list_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Users Data Schema

* _id: user Id
* state: state abbreviation
* createdDate: when the user created their account
* lastLogin: last time the user was recorded logging in to the app
* role: constant value set to 'CONSUMER'
* active: indicates if the user is active; only Fetch will de-activate an account with this flag

In [9]:
# Uploading and converting the json file into a dataframe
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Using pandas.read_json with lines=True to handle JSONL files
user_df = pd.read_json(filename, lines=True)
user_df.head()

Saving users.json to users (1).json


,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [10]:
# Data transformation
user_df['id'] = user_df['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else x)
user_df.drop(columns=['_id'], inplace=True)

user_df = user_df.map(extract_date)
epoch_list = ['createdDate', 'lastLogin']
user_df = convert_epoch_to_datetime(user_df, epoch_list)
user_df.head()

,active,createdDate,lastLogin,role,signUpSource,state,id,createdDate_datetime,lastLogin_datetime
0,True,1609687444800,1.609688e+12,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
1,True,1609687444800,1.609688e+12,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
2,True,1609687444800,1.609688e+12,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
3,True,1609687530554,1.609688e+12,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597
4,True,1609687444800,1.609688e+12,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858


In [21]:
user_df.columns

Index(['active', 'createdDate', 'lastLogin', 'role', 'signUpSource', 'state',
       'id', 'createdDate_datetime', 'lastLogin_datetime'],
      dtype='object')

In [11]:
# renaming for readability
user_df = user_df.rename(columns={'id': 'user_id', 'createdDate': 'createdDate_epoch', 'lastLogin': 'lastLogin_epoch',
                                  'createdDate_datetime': 'createdDate', 'lastLogin_datetime': 'lastLogin'})
user_df.head()

,active,createdDate_epoch,lastLogin_epoch,role,signUpSource,state,user_id,createdDate,lastLogin
0,True,1609687444800,1.609688e+12,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
1,True,1609687444800,1.609688e+12,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
2,True,1609687444800,1.609688e+12,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
3,True,1609687530554,1.609688e+12,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597
4,True,1609687444800,1.609688e+12,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858


In [12]:
user_df.to_csv('users.csv', index=False)

# Step 2: Download the CSV file
files.download('users.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Brand Data Schema

* _id: brand uuid
* barcode: the barcode on the item
* brandCode: String that corresponds with the brand column in a partner product file
* category: The category name for which the brand sells products in
* categoryCode: The category code that references a BrandCategory
* cpg: reference to CPG collection
* topBrand: Boolean indicator for whether the brand should be featured as a 'top brand'
* name: Brand name


In [13]:
# Uploading and converting the json file into a dataframe
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Use pandas.read_json with lines=True to handle JSONL files
brand_df = pd.read_json(filename, lines=True)
brand_df.head()

Saving brands.json to brands.json


,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [14]:
# Data transformation
brand_df['id'] = brand_df['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else x)
brand_df.drop(columns=['_id'], inplace=True)

df_extracted = brand_df['cpg'].apply(lambda x: pd.Series(x) if isinstance(x, dict) else pd.Series())

# Combining the extracted columns with the original DataFrame
dict_column = 'cpg'
for col in df_extracted.columns:
    brand_df[f"{dict_column}_{col}"] = df_extracted[col]

brand_df.head()

,barcode,category,categoryCode,cpg,name,topBrand,brandCode,id,cpg_$id,cpg_$ref
0,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,601ac115be37ce2ead437551,{'$oid': '601ac114be37ce2ead437550'},Cogs
1,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,601c5460be37ce2ead43755f,{'$oid': '5332f5fbe4b03c9a25efd0ba'},Cogs
2,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,{'$oid': '601ac142be37ce2ead437559'},Cogs
3,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,{'$oid': '601ac142be37ce2ead437559'},Cogs
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,{'$oid': '5332fa12e4b03c9a25efd1e7'},Cogs


In [15]:
brand_df['cpg_$id'] = brand_df['cpg_$id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else x)
brand_df.head()

,barcode,category,categoryCode,cpg,name,topBrand,brandCode,id,cpg_$id,cpg_$ref
0,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
1,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs
2,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs
3,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs


In [40]:
brand_df['barcode'].nunique()

1160

In [16]:
brand_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   barcode       1167 non-null   int64  
 1   category      1012 non-null   object 
 2   categoryCode  517 non-null    object 
 3   cpg           1167 non-null   object 
 4   name          1167 non-null   object 
 5   topBrand      555 non-null    float64
 6   brandCode     933 non-null    object 
 7   id            1167 non-null   object 
 8   cpg_$id       1167 non-null   object 
 9   cpg_$ref      1167 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 91.3+ KB


In [17]:
# renaming for readability and droping a column
brand_df = brand_df.drop(columns=['cpg'])
brand_df = brand_df.rename(columns={'id': 'brand_id', 'cpg_$id': 'cpg_id', 'cpg_$ref': 'cpg_ref'})
brand_df.head()

,barcode,category,categoryCode,cpg,name,topBrand,brandCode,brand_id,cpg_id,cpg_$ref
0,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
1,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs
2,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs
3,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs


In [20]:
brand_df.to_csv('brands.csv', index=False)

# Step 2: Download the CSV file
files.download('brands.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
brand_df.columns

Index(['barcode', 'category', 'categoryCode', 'name', 'topBrand', 'brandCode',
       'brand_id', 'cpg_id', 'cpg_ref'],
      dtype='object')

# Checking for matching keys

In [54]:
receipt_list_df['rewardsReceiptItemList_barcode'] = pd.to_numeric(df['rewardsReceiptItemList_barcode'], errors='coerce').astype('Int64')


In [51]:
receipt_list_df.columns

Index(['userId', 'id', 'rewardsReceiptItemList_barcode',
       'rewardsReceiptItemList_description',
       'rewardsReceiptItemList_finalPrice', 'rewardsReceiptItemList_itemPrice',
       'rewardsReceiptItemList_needsFetchReview',
       'rewardsReceiptItemList_partnerItemId',
       'rewardsReceiptItemList_preventTargetGapPoints',
       'rewardsReceiptItemList_quantityPurchased',
       'rewardsReceiptItemList_userFlaggedBarcode',
       'rewardsReceiptItemList_userFlaggedNewItem',
       'rewardsReceiptItemList_userFlaggedPrice',
       'rewardsReceiptItemList_userFlaggedQuantity',
       'rewardsReceiptItemList_originalMetaBriteBarcode',
       'rewardsReceiptItemList_originalMetaBriteDescription',
       'rewardsReceiptItemList_pointsNotAwardedReason',
       'rewardsReceiptItemList_pointsPayerId',
       'rewardsReceiptItemList_rewardsGroup',
       'rewardsReceiptItemList_rewardsProductPartnerId',
       'rewardsReceiptItemList_brandCode',
       'rewardsReceiptItemList_competi

In [55]:
# Merge DataFrames based on different keys - Barcode
merged_df_barcode = pd.merge(receipt_list_df, brand_df, left_on='rewardsReceiptItemList_barcode', right_on='barcode', how='inner')
match_count = merged_df_barcode.shape[0]

# Logically this is a unique integer combination for every product so in an ideal world there should be a match

In [56]:
match_count

0

In [60]:
# cpg_id to pointsPayerId
merged_df_barcode = pd.merge(receipt_list_df, brand_df, left_on='rewardsReceiptItemList_pointsPayerId', right_on='cpg_$id', how='inner')
merged_df_barcode.shape[0]

13502

In [62]:
brand_df.shape[0] # There seem to be duplication

1167

In [66]:
# ProductpartnerId to cpg_id
merged_df = pd.merge(receipt_list_df, brand_df, left_on='rewardsReceiptItemList_rewardsProductPartnerId', right_on='cpg_$id', how='inner')
merged_df.shape[0] # There seem to be duplication

21014

In [67]:
# brandCode to brandCode
merged_df = pd.merge(receipt_list_df, brand_df, left_on='rewardsReceiptItemList_brandCode', right_on='brandCode', how='inner')
merged_df.shape[0] # There seem to be duplication

# Ideally this should match but given that brandcode are just brand names in string there could be human error such as typo.

243608

In [68]:
!pip install duckdb # package to build in-memory db for sql querying

In [ ]:
import duckdb

# in-memory DuckDB connection
con = duckdb.connect()
# Registering DataFrames as DuckDB tables (a virtual table)
con.register('receipts', receipt_df)
con.register('receipt_list_df', receipt_list_df)
con.register('brand_df', brand_df)

# con.execute("SELECT * FROM receipts").df()


In [74]:
# While there are no matches in barcode between brands and receipt_item_list, logically barcode are unique
# and in an ideal world I am assuming this will be a key with matches
con.execute(
'''select count(distinct barcode), count(distinct rewardsReceiptItemList_barcode)
from brand_df left join receipt_list_df on brand_df.barcode = receipt_list_df.rewardsReceiptItemList_barcode''').df()


,count(DISTINCT barcode),count(DISTINCT rewardsReceiptItemList_barcode)
0,1160,0


In [81]:
con.execute(
'''select count(distinct barcode), count(distinct rewardsReceiptItemList_userFlaggedBarcode)
from brand_df left join receipt_list_df on brand_df.barcode = receipt_list_df.rewardsReceiptItemList_userFlaggedBarcode''').df()


,count(DISTINCT barcode),count(DISTINCT rewardsReceiptItemList_userFlaggedBarcode)
0,1160,0


In [85]:

con.execute(
'''select count(distinct barcode), count(distinct rewardsReceiptItemList_originalMetaBriteBarcode)
from brand_df left join receipt_list_df on cast(brand_df.barcode as char) = receipt_list_df.rewardsReceiptItemList_originalMetaBriteBarcode''').df()


,count(DISTINCT barcode),count(DISTINCT rewardsReceiptItemList_originalMetaBriteBarcode)
0,1160,0


In [78]:
con.execute(
'''select count(distinct brand_df.Id), count(distinct receipt_list_df.id)
from brand_df left join receipt_list_df on brand_df.Id = receipt_list_df.id''').df()


,count(DISTINCT brand_df.Id),count(DISTINCT receipt_list_df.id)
0,1167,0


In [79]:
con.execute(
'''select count(distinct brand_df.cpg_$id), count(distinct receipt_list_df.rewardsReceiptItemList_partnerItemId)
from brand_df left join receipt_list_df on brand_df.cpg_$id = receipt_list_df.rewardsReceiptItemList_partnerItemId''').df()


,"count(DISTINCT brand_df.""cpg_$id"")",count(DISTINCT receipt_list_df.rewardsReceiptItemList_partnerItemId)
0,196,0


In [80]:
con.execute(
'''select count(distinct brand_df.cpg_$id), count(distinct receipt_list_df.rewardsReceiptItemList_pointsPayerId)
from brand_df left join receipt_list_df on brand_df.cpg_$id = receipt_list_df.rewardsReceiptItemList_pointsPayerId''').df()


,"count(DISTINCT brand_df.""cpg_$id"")",count(DISTINCT receipt_list_df.rewardsReceiptItemList_pointsPayerId)
0,196,6


In [86]:
con.execute(
'''select count(distinct brand_df.cpg_$id), count(distinct receipt_list_df.rewardsReceiptItemList_rewardsProductPartnerId)
from brand_df left join receipt_list_df on brand_df.cpg_$id = receipt_list_df.rewardsReceiptItemList_rewardsProductPartnerId''').df()
# Some match found for rewardsReceiptItemList_rewardsProductPartnerId with cpg_id

,"count(DISTINCT brand_df.""cpg_$id"")",count(DISTINCT receipt_list_df.rewardsReceiptItemList_rewardsProductPartnerId)
0,196,7


In [87]:
con.execute(
'''select count(distinct brand_df.brandCode), count(distinct receipt_list_df.rewardsReceiptItemList_brandCode)
from brand_df left join receipt_list_df on brand_df.brandCode = receipt_list_df.rewardsReceiptItemList_brandCode''').df()


,count(DISTINCT brand_df.brandCode),count(DISTINCT receipt_list_df.rewardsReceiptItemList_brandCode)
0,897,6


List of keys and their Matches


*   receipts.receipt_id = receiptItemList.receipt_id
*   receipts.user_id = receiptItemList.user_id
*   receipts.user_id = users.user_id
*  receiptItemList.barcode = brands.barcode
*  receiptItemList.brandCode = brands.brandCode
*  receiptItemList.rewardsProductPartnerId = brands.cpg_idv (needs validation from SME)

